In [33]:
!pip install pandas_datareader
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install stocktrends

In [34]:
import pandas_datareader.data as pdr
import datetime
# https://github.com/ChillarAnand/stocktrends
from stocktrends import Renko

In [35]:
# Download historical data for required stocks
ticker = "AAPL"
ohlcv = pdr.get_data_yahoo(ticker,datetime.date.today()-datetime.timedelta(364),datetime.date.today())

In [36]:
df = ohlcv.copy()

In [37]:
ohlcv.shape

(251, 6)

In [38]:
ohlcv.head(4)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-12-20,162.110001,155.300003,160.399994,156.830002,64773000.0,154.508499
2018-12-21,158.160004,149.630005,156.860001,150.729996,95744600.0,148.498795
2018-12-24,151.550003,146.589996,148.149994,146.830002,37169200.0,144.656540
2018-12-26,157.229996,146.720001,148.300003,157.169998,58582500.0,154.843475


In [39]:
ohlcv.tail(4)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-12-13,275.299988,270.929993,271.459991,275.149994,33396900.0,275.149994
2019-12-16,280.790009,276.980011,277.000000,279.859985,32046500.0,279.859985
2019-12-17,281.769989,278.799988,279.570007,280.410004,28539600.0,280.410004
2019-12-18,281.899994,279.119995,279.799988,279.739990,29007100.0,279.739990


In [40]:
def ATR(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Adj Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Adj Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df2 = df.drop(['H-L','H-PC','L-PC'],axis=1)
    return df2

In [41]:
def renko_DF(DF):
    "function to convert ohlc data into renko bricks"
    df = DF.copy()
    df.reset_index(inplace=True)
    df = df.iloc[:,[0,1,2,3,5,6]]
    df.rename(columns = {"Date" : "date", "High" : "high","Low" : "low", "Open" : "open","Adj Close" : "close", "Volume" : "volume"}, inplace = True)
    df2 = Renko(df)
    df2.brick_size = round(ATR(DF,120)["ATR"][-1],0)
    renko_df = df2.get_ohlc_data() #if get_bricks() does not work try using get_ohlc_data() instead
    #renko_df = df2.get_ohlc_data()
    return renko_df

In [43]:
renko_df_result = renko_DF(ohlcv)

In [45]:
renko_df_result.tail(60)

,date,open,high,low,close,uptrend
0,2018-12-20,148.0,152.0,148.0,152.0,True
1,2019-01-03,148.0,148.0,144.0,144.0,False
2,2019-01-16,148.0,152.0,148.0,152.0,True
3,2019-01-30,152.0,156.0,152.0,156.0,True
4,2019-01-30,156.0,160.0,156.0,160.0,True
5,2019-02-01,160.0,164.0,160.0,164.0,True
6,2019-02-04,164.0,168.0,164.0,168.0,True
7,2019-02-25,168.0,172.0,168.0,172.0,True
8,2019-03-11,172.0,176.0,172.0,176.0,True
9,2019-03-14,176.0,180.0,176.0,180.0,True
